In [45]:
import psycopg2
import db_config as creds


def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    #print (conn_string)
    
    #conn = psycopg2.connect(conn_string)
    #conn = psycopg2.connect(dbname='postgres', user='postgres', host='postgres-db.c3yvclp4kwun.eu-west-2.rds.amazonaws.com', port=5432, password='dev6o2VVxXzxDSDwfyE5')
    
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

def disconnect(conn,cursor):
    
    conn.close()

In [46]:
# Connecting to DB
conn, cursor = connect()

insert_image = """
    INSERT INTO images (filename) VALUES('UA502')       
    """

# Execute SQL Command and commit to DB
cursor.execute(insert_image)
conn.commit()

# Disconnect from DB
disconnect(conn, cursor)

Connected!
